In [1]:
from pyspark.sql import Row
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark import sql
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = sql.SQLContext(sc)


In [2]:
from pyspark.sql.functions import lit

def process(t):
  key = t[0]
  text = t[1]
  rows = text.split("\n")
  last_key = ""
  prev_key = ""
  val = ""
  result = []
  processed_cont=True
  emit_value = False
  dic = {}
  
  for row in rows:
    if len(row) < 12:
      if processed_cont:
        #if emit_value:
        #  dic[re.sub(' +',' ',last_key.strip(' \t\n\r'))] = re.sub(' +',' ',val.strip('\t\n\r'))
          #result += [(key, {: })]
        #  val = ""
        prev_key += row
        processed_cont = False
        #emit_value = False
      else:
        prev_key+=" "+row
    elif row[12] == ':':
      if emit_value:
        #result += [(key, {re.sub(' +',' ',last_key.strip(' \t\n\r')): re.sub(' +',' ',val.strip('\t\n\r'))})]
        dic[re.sub(' +',' ',last_key.strip(' \t\n\r'))] = re.sub(' +',' ',val.strip('\t\n\r'))
        val = ""
      if processed_cont:
        last_key = last_key =prev_key+row[:12]
      else:
        last_key =prev_key+" "+row[:12]
      prev_key = ""
      val+=row[13:]
      #result+=[((key,last_key), row[13:])]
      processed_cont = True
      emit_value = True
    else:
      val+=row
      emit_value = True
      processed_cont = True
      
  dic[re.sub(' +',' ',last_key.strip(' \t\n\r'))] = re.sub(' +',' ',val.strip('\t\n\r'))
  #result += [(key, {re.sub(' +',' ',last_key.strip(' \t\n\r')): re.sub(' +',' ',val.strip('\t\n\r'))})]
  return (key, dic)

def generate_row(rdd_in, dict_keys):
  r = [rdd_in[0]]
  for k in dict_keys:
    try:
      val = rdd_in[1][k]
      if val == "Not Available":
        r+=[None]
      else:
        r+=[val]
    except:
      print(rdd_in[0], ': ', k, " not found")
      r+=[None]
  return r

In [3]:
data = df.map(process)
data.take(1)

Out[ 7 ]: 
[('dbfs:/data-mining/nsf-abstract/original/awd_1992_00/a9200001.txt',
 {'Type': ' Award',
 'Investigator': ' Pierre J. Flament pierre@soest.hawaii.edu (Principal Investigator current)',
 'Expected Total Amt.': ' $138674 (Estimated)',
 'Latest Amendment Date': ' January 14, 1993 ',
 'Award Number': ' 9200001',
 'Sponsor': ' U of Hawaii Manoa\t 2530 Dole Street\t Honolulu, HI 968222225 808/956-7800',
 'Prgm Manager': ' Neil R. Andersen \t OCE DIVISION OF OCEAN SCIENCES \t GEO DIRECTORATE FOR GEOSCIENCES ',
 'File': ' a9200001',
 'Abstract': ' A station to receive, archive and process high resolution images from the NOAA polar-orbiting satellites will be installed on board the R/V Thompson and operated during the JGOFS cruises. Images of sea surface temperature will be generated and will be accessible on the ship to support field studies requiring real- time satellite coverage. After post-cruise quality control, the images will be published on CD-ROMs for distribution to all JGOFS investigators. A domain extending from 12.5oS to 12.5oN and 130oW to 145oW will be covered. Processes that can be observed in this domain include the upwelling tongue along the equator and the Intertropical Convergence Zone at 8oN. The data will be used for statistical studies of the formation, evolution and decay of surface temperature patterns, as well as for process-oriented studies for which simultaneous shipboard sampling is available.',
 'Program Ref': ' 1577,',
 'Title': ' Acquisition and Processing of AVHRR Images During JGOFS',
 'Expires': ' November 30, 1994 (Estimated)',
 'NSF Program': ' 1670 CHEMICAL OCEANOGRAPHY',
 'NSF Org': ' OCE ',
 'Fld Applictn': ' 0204000 Oceanography 44 Physical & Chemical Oceanography ',
 'Start Date': ' December 1, 1991 ',
 'Award Instr.': ' Continuing grant '})]

In [4]:
import re
from pyspark.sql import Row

data = df.map(process)
#.map(lambda x: ((x[0][0],re.sub(' +',' ',x[0][1].strip(' \t\n\r'))), re.sub(' +',' ',x[1].strip('\t\n\r'))))\
#.map(lambda x: (x[0][0], {x[0][1]: x[1].strip(" ")}))\
#.reduceByKey(lambda x, y: {**x, **y}).cache()
#.collect()
#data
#data.take(1)[0][1].keys()
keys = list(map(lambda a: list(a[1].keys()), data.take(1)))[0]

table = data.map(lambda x: generate_row(x, keys))\
.toDF(["path"]+keys)

table.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/data-mining/nsf-abstract/cleaned/second/1")

#%python
#display(table)
#.reduceByKey(lambda x,y: x+y)\
#.toDF("Award Number",)


In [5]:
table

Out[ 173 ]: 
[['dbfs:/data-mining/nsf-abstract/original/awd_1992_54/a9254859.txt',
 '$7500 (Estimated)',
 '9254859',
 '0000099 Other Applications NEC 99 Other Sciences NEC',
 'Jim Hill Junior HS 1000 6th Street SW Minot, ND 587014520 701/857-4477',
 'September 30, 1995 (Estimated)',
 'October 1, 1992',
 'a9254859',
 'Standard Grant',
 'ESI',
 'Award',
 'Rose Marie Smith ESI DIV OF ELEM SECONDARY & INFORMAL SCI ED EHR DIRECT FOR EDUCATION AND HUMAN RESOURCES',
 'Not Available',
 'September 27, 1992',
 '7345 PRES AWARDS FOR EXCEL IN SCIEN',
 'Presidential Awards for Excellence in Science and Mathematics Teaching (Secondary)',
 '',
 'Janet Washek (Principal Investigator current)'],
 ['dbfs:/data-mining/nsf-abstract/original/awd_1992_54/a9254857.txt',
 '$7500 (Estimated)',
 '9254857',
 '0000099 Other Applications NEC 99 Other Sciences NEC',
 'Daniels Middle School 2816 Oberlin Road Raleigh, NC 276081220 919/882-4860',
 'September 30, 1995 (Estimated)',
 'October 1, 1992',
 'a9254857',
 'Standard Grant',
 'ESI',
 'Award',
 'Rose Marie Smith ESI DIV OF ELEM SECONDARY & INFORMAL SCI ED EHR DIRECT FOR EDUCATION AND HUMAN RESOURCES',
 'Not Available',
 'September 27, 1992',
 '7345 PRES AWARDS FOR EXCEL IN SCIEN',
 'Presidential Awards for Excellence in Science and Mathematics Teaching (Secondary)',
 '',
 'Julia A. Cazin (Principal Investigator current)'],
 ['dbfs:/data-mining/nsf-abstract/original/awd_1992_54/a9254514.txt',
 '$7500 (Estimated)',
 '9254514',
 '0000099 Other Applications NEC 99 Other Sciences NEC',
 'Old Greenwich School 285 Sound Beach Avenue Old Greenwich, CT 068701607 203/637-0150',
 'September 30, 1995 (Estimated)',
 'October 1, 1992',
 'a9254514',
 'Standard Grant',
 'ESI',
 'Award',
 'Rose Marie Smith ESI DIV OF ELEM SECONDARY & INFORMAL SCI ED EHR DIRECT FOR EDUCATION AND HUMAN RESOURCES',
 'Not Available',
 'September 27, 1992',
 '7345 PRES AWARDS FOR EXCEL IN SCIEN',
 'Presidential Awards for Excellence in Science and Mathematics Teaching',
 '',
 'Linda J. McMahan (Principal Investigator current)'],
 ['dbfs:/data-mining/nsf-abstract/original/awd_1992_54/a9254165.txt',
 '$79946 (Estimated)',
 '9254165',
 '0000099 Other Applications NEC 21 Mathematics',
 'U of Texas San Antonio 6900 North Loop 1604 West San Antonio, TX 78249 210/458-4340',
 'May 31, 1996 (Estimated)',
 'July 1, 1993',
 'a9254165',
 'Standard Grant',
 'DUE',
 'Award',
 'Herbert Levitan DUE DIVISION OF UNDERGRADUATE EDUCATION EHR DIRECT FOR EDUCATION AND HUMAN RESOURCES',
 "The Mathematics Improvement Project (MIP) represents a concentrated effort to increase the numbers of underrepresented students majoring in science, mathematics, or engineering at the University of Texas at San Antonio (UTSA) by focusing on their successful completion of College Algebra and Pre- calculus. The project is incorporating programmatic structures that facilitate faculty training, curricular design, and implementation of pedagogical practices designed to both meet the special needs of underrepresented students and to promote critical thinking and mastery of mathematical concepts. Four faculty within the Division of Mathematics, Computer Science, and Statistics (MCSS) at UTSA are working as MIP faculty and are designing and implementing a culture-sensitive learning environment as well as a non-traditional instructional curriculum to insure that students are prepared to work in the global community in the 21st century. Although the program is open to all students, underrepresented students enrolling in College Algebra and Pre-calculus are being targeted. Through collaboration with the university's Student Services offices and by formalizing ties with local schools and community colleges, this program is implementing an effective system of recruitment, retention, and successful completion of the MIP mathematics sequence for underrepresented students. MIP's pipeline is generating a new momentum which allows students to build confidence and to achieve an incremental gain in 

In [6]:
%python
display(table)

path,Fld Applictn,Type,Award Number,NSF Program,Expires,Award Instr.,Sponsor,Expected Total Amt.,Prgm Manager,File,Program Ref,Investigator,Latest Amendment Date,Abstract,Start Date,NSF Org,Title
dbfs:/data-mining/nsf-abstract/original/awd_1992_20/a9220001.txt,0000099 Other Applications NEC 61 Life Science Biological,Award,9220001,1136 SIGNAL TRANSDCTN/CELL REGULATN,"December 31, 1995 (Estimated)",Continuing grant,"NYU Medical Center 550 First Avenue New York, NY 100166481 212/263-6693",$300000 (Estimated),Barbara K. Zain MCB DIV OF MOLECULAR AND CELLULAR BIOSCIENCE BIO DIRECT FOR BIOLOGICAL SCIENCES,a9220001,"9119,9183,BIOT,",Marie E. Monaco mem6@nyu.edu (Principal Investigator current),"April 3, 1995","Hormone-stimulated hydrolysis of phosphatidyl inositol bis- phosphate is followed by conversion of phosphatidyl inositol to phosphatidyl inositol bis-phosphate. Studies indicate that lipid from subcellular organelles in addition to the plasma membrane must be accessible in order to account for the large amount of PtdIns that is often involved in the cycling response. To date, two mechanisms have been proposed to explain this: (1) A PtdIns transfer protein facilitates the transfer of intracellular PtdIns to the plasma membrane, and (2) The bulk of the PtdIns metabolized is hydrolyzed directly, perhaps as a result of the rise in intracellular free calcium, and is not involved in the generation of inositol tris phosphate second messengers or directly associated with a hormone-receptor complex. A third possibility is suggested: (3) Internal membrane PtdIns is accessed as a result of receptor sequestration and recycling through the interior of the cell. A series of experiments have been devised to test this third possibility. They involve inhibition of internalization by potassium depletion as well as by receptor mutation. As a corollary to this hypothesis, compensatory resynthesis of PtdIns need not take place at the plasma membrane. The subcellular location of the PtdIns synthetase enzyme will be determined by immunocytochemistry. %%% One important signaling pathway in the response of cells to hormones involves the breakdown of a membrane component known as phosphatidylinositol bis-phosphate to produce further signal molecules. To maintain a cell's ability to respond to hormone, this material must be resynthesized. The goal of this research is to determine where within the cell resynthesis takes place and how it is controlled in coordination with hormonal stimulation of the cell. The results of this research will contribute important new information on how cells respond to hormones and other external signals.","July 1, 1993",MCB,Cyclic Phosphatidylinositol Metabolism and Resynthesis
dbfs:/data-mining/nsf-abstract/original/awd_1992_20/a9220004.txt,0000099 Other Applications NEC 42 Geological Sciences,Award,9220004,1573 PETROLOGY AND GEOCHEMISTRY,"January 31, 1994 (Estimated)",Standard Grant,"Columbia University 1210 Amsterdam Avenue; MC 2205 New York, NY 10027 212/854-6851",$40000 (Estimated),Jonathan H. Fink EAR DIVISION OF EARTH SCIENCES GEO DIRECTORATE FOR GEOSCIENCES,a9220004,,Alan Zindler (Principal Investigator current),"January 11, 1993","The principal goals of this research are to investigate Re- Os systematics in the upper mantle and the nature of processes that produce Os isotopic variations in the mantle. To achieve these goals, the P.I.s will implement a two-phase research program. First, they will expand the characterization of Os isotope ratios in OIBs by analyzing selected basalts from Iceland, Hawaii, St. Helena and the Canary Islands. Second, they will investigate the extent to which assimilation of altered crust may be important in determining the Os isotopic composition of OIBs by comparing Re-Os systematics in differentiated and undifferentiated basalts, and by direct characterization of altered submarine basalt sections.","February 1, 1993",EAR,Re-Os Systematics in the Earth's Mantle: Evidence from Mantle-Deri